## מספר המדבקות שאינן במקומן

In [ ]:
def _calculate_heuristic(self):
    """Calculate heuristic based on number of misplaced stickers"""
    target = {
        'U': [0, 0, 0, 0], 'R': [1, 1, 1, 1], 'F': [2, 2, 2, 2],
        'D': [5, 5, 5, 5], 'L': [4, 4, 4, 4], 'B': [3, 3, 3, 3]
    }
    
    misplaced = 0
    for face in self.cube:
        for i in range(4):
            if self.cube[face][i] != target[face][i]:
                misplaced += 1
    
    self.phi = misplaced


בניגוד לגרסה הפשוטה (שסופרת רק את מספר המדבקות שלא במקומן), כאן זו דרך שמחשבת פונקציית הֵאוּרִיסְטִיקָה מורכבת יותר, שמבוססת על שילוב של כמה פקטורים:

_corner_heuristic – מצב הפינות (אם הפינה במיקום הנכון ואם היא בסיבוב נכון).

_edge_heuristic – מצב הקצוות.

_cross_heuristic – האם יש צורת “צלב” במרכז הפאה העליונה והתחתונה.

_center_heuristic – בדיקה אם המרכזים נכונים (בדרך כלל הם לא זזים, אז זו בדיקה נוספת).

In [ ]:
# ==============================================================================
def _calculate_heuristic(self):
    """Calculate advanced heuristic combining multiple factors"""
    # Multi-component heuristic for better convergence
    corner_score = self._corner_heuristic()
    edge_score = self._edge_heuristic() 
    cross_score = self._cross_heuristic()
    center_score = self._center_heuristic()
    
    # Weighted combination - prioritize solving order (cross, then corners, then edges)
    self.phi = (cross_score * 3.0 + corner_score * 2.0 + edge_score * 1.5 + center_score * 0.5)

def _corner_heuristic(self):
    """Score based on corner piece positions and orientations"""
    score = 0
    corner_positions = [
        # (face, row, col) for each corner
        [('U',0,0), ('L',0,0), ('B',0,2)], [('U',0,2), ('B',0,0), ('R',0,2)],
        [('U',2,0), ('F',0,0), ('L',0,2)], [('U',2,2), ('R',0,0), ('F',0,2)],
        [('D',0,0), ('L',2,2), ('F',2,0)], [('D',0,2), ('F',2,2), ('R',2,0)],
        [('D',2,0), ('B',2,0), ('L',2,0)], [('D',2,2), ('R',2,2), ('B',2,2)]
    ]
    
    target_corners = [
        [0,4,3], [0,3,1], [0,2,4], [0,1,2],  # Top corners
        [5,4,2], [5,2,1], [5,3,4], [5,1,3]   # Bottom corners
    ]
    
    for i, corner_pos in enumerate(corner_positions):
        current = [self.cube[face][row][col] for face, row, col in corner_pos]
        target = target_corners[i]
        
        # Position penalty: is corner in right position?
        if set(current) != set(target):
            score += 3  # Wrong corner entirely
        else:
            # Orientation penalty: corner in right place but twisted?
            if current != target:
                score += 1  # Right corner, wrong orientation
    
    return score

def _edge_heuristic(self):
    """Score based on edge piece positions and orientations"""
    score = 0
    edge_positions = [
        # Top edges
        [('U',0,1), ('B',0,1)], [('U',1,2), ('R',0,1)], 
        [('U',2,1), ('F',0,1)], [('U',1,0), ('L',0,1)],
        # Middle edges  
        [('F',1,0), ('L',1,2)], [('F',1,2), ('R',1,0)],
        [('B',1,0), ('R',1,2)], [('B',1,2), ('L',1,0)],
        # Bottom edges
        [('D',0,1), ('F',2,1)], [('D',1,2), ('R',2,1)],
        [('D',2,1), ('B',2,1)], [('D',1,0), ('L',2,1)]
    ]
    
    target_edges = [
        [0,3], [0,1], [0,2], [0,4],  # Top edges
        [2,4], [2,1], [3,1], [3,4],  # Middle edges  
        [5,2], [5,1], [5,3], [5,4]   # Bottom edges
    ]
    
    for i, edge_pos in enumerate(edge_positions):
        current = [self.cube[face][row][col] for face, row, col in edge_pos]
        target = target_edges[i]
        
        # Position penalty
        if set(current) != set(target):
            score += 2  # Wrong edge entirely
        else:
            # Orientation penalty  
            if current != target:
                score += 1  # Right edge, flipped
    
    return score

def _cross_heuristic(self):
    """Score for having cross patterns on top and bottom faces"""
    score = 0
    
    # Top cross (white)
    top_cross = [self.cube['U'][0][1], self.cube['U'][1][0], 
                self.cube['U'][1][2], self.cube['U'][2][1]]
    if not all(piece == 0 for piece in top_cross):
        score += 8  # Heavy penalty for missing top cross
        
    # Bottom cross (yellow)  
    bottom_cross = [self.cube['D'][0][1], self.cube['D'][1][0],
                self.cube['D'][1][2], self.cube['D'][2][1]]
    if not all(piece == 5 for piece in bottom_cross):
        score += 4  # Medium penalty for missing bottom cross
        
    return score

def _center_heuristic(self):
    """Score based on center pieces (should always be correct on 3x3x3)"""
    score = 0
    centers = {'U': 0, 'R': 1, 'F': 2, 'D': 5, 'L': 4, 'B': 3}
    
    for face, target_color in centers.items():
        if self.cube[face][1][1] != target_color:
            score += 10  # Centers should never move on 3x3x3!
            
    return score
# =============================================================================